# Grupo 3 - Perceptron Modeling (Exercício 1)

---------------------------------------------

### Participants:
 - Francielle Vargas - 9527629
 - Lucas Nunes Sequeira - 9009642
 - Emanuel Huber - 12110113

#### Date: 19/08/2021

---------------------------------------------

#### Descrição e Instrução

Este notebook foi feito para a disciplina SCC5809 - Redes Neurais

No notebook contém

1. A implementação da classe **Perceptron**
2. Um exemplo de aquecimento (Problema do operador AND)
3. O problema do A e A invertido

Para utilizá-lo, basta executar todas as células deste notebook.

_link de acesso ao colab: https://colab.research.google.com/drive/1SK1j-izxdEi8fsWOMndYXWLARkTLfrk2?usp=sharing_

### Libs

In [1]:
# Install Libs
!pip install numpy==1.19.5
!pip install tqdm==4.62.0
!pip install plotly==4.4.1

In [2]:
# Math lib
import numpy as np

# Log lib
from tqdm.auto import tqdm

# Visualization Lib
import plotly.express as px

### Activation function

In [3]:
def f_step1(x: float) -> float:
  '''Step Function
     Given a x, return 1 if x > 0, otherwise returns 0
  '''

  if x > 0.0:
    return 1.0
  else:
    return 0.0

In [4]:
def f_step2(x: float) -> float:
  '''Step Function
     Given a x, return 1 if x > 0, otherwise returns -1
  '''

  if x > 0.0:
    return 1.0
  else:
    return -1.0

### Perceptron Class

In [5]:
class Perceptron():

  def __init__(self, input_size: int = 2, init_rule: str = 'zero', activation = f_step1):
    '''Initializes Perceptron
    
      Params:
        input_size (int): size of input data
        init_rule (str): initialization parameter to set initial weight values
        activation (func): function to apply activation step
      '''

    # Perceptron size
    self.input_size = input_size
    self.size = input_size + 1

    # Initialize weights
    self.init_weights(init_rule)

    # Activation function
    self.activation = np.vectorize(activation)

    # Health check
    self._health_check()
  
  def _health_check(self):
    '''Perform a health check prediction'''

    # 1D (input_size)
    X = np.random.rand(self.input_size)
    self.predict(X)

    # 2D (4 items, input_size)
    X = np.random.rand(4, self.input_size)
    self.predict(X)

  def init_weights(self, init_rule: str = 'zero'):
    '''Initialize weights
    
      Params:
        init_rule (str): initialization parameter to set initial weight values
    '''

    # Assure initializarion rule
    assert init_rule in ['zero', 'rand'], "'init_rule' must be zero or rand"

    if init_rule == 'zero':
      # Zero values
      self.weights = np.zeros(self.size)
    
    elif init_rule == 'rand':
      # Random values in [-0.5, 0,5]
      self.weights = np.random.rand(self.size) - 0.5

  def set_weights(self, weights: np.ndarray):
    '''Update weights

      Params:
        weights (np.ndarray): array of weights
    '''

    # Assure perceptron size equals given weights size
    assert self.size == len(weights), f"Perceptron size ({self.size}) != weights size ({len(weights)})"

    # Update weights
    self.weights = weights.copy()

  def get_weights(self) -> np.ndarray:
    '''Return a copy of current weights'''

    # Return weights
    return self.weights.copy()

  def _get_item(self, X: np.ndarray, y: np.ndarray):
    '''Generator of single item from X and y input data'''

    # X and y lengths must match
    assert len(X) == len(y), f"X (len = {len(X)}) and y (len = {len(y)}) lengths must match"

    for item_idx in range(len(X)):

      # Return item
      yield X[item_idx], y[item_idx]

  def _apply_delta_rule(self, x: np.ndarray, loss: float, learning_rate: float):
    '''Apply delta rule to update weights, ie:

       delta(w_i) = lr * loss * x_i (delta rule)

      Params:
        x (np.ndarray): 1D dimensional item
        loss (float): loss calculated with the given item
        learning_rate (float): learning rate parameter
    '''

    # Add bias term
    x = self._add_bias_term(x, is_batch=False)

    # Get a copy of weights
    current_weights = self.get_weights()

    # Iterate over all x features
    for i, x_i in enumerate(x):
      
      # Delta rule
      current_weights[i] += learning_rate * loss * x_i

    # Update weights
    self.set_weights(current_weights)
    
  def fit(self, X: np.ndarray, y: np.ndarray, learning_rate: float = 0.5,
          max_epochs: int = 10, stop_threshold: float = 1e-6) -> dict:
    '''Train method
    
      Params:
        X (np.ndarray): a array of inputs, each input must match perceptron given input_size (number of features)
        y (np.ndarray): a array of target values (labels)
        learning_rate (float): hyperparameter to be used on delta rule learning
        max_epochs (int): number of maximum epochs to iterate
        stop_threshold (float): number to be used to stop training if epoch loss is lower than

      Returns:
        history (dict): A dictionary containing training data over training as epoch loss
    '''

    # Assure X and y has same size
    assert len(X) == len(y), "X and y must match sizes."

    # Epoch iterator
    iterator = tqdm(range(max_epochs))

    # Num items
    num_items = len(y)

    # History of train
    history = {'loss': []}

    for epoch in iterator:

      # Log epoch
      iterator.set_description(f'Epoch {epoch+1}/{max_epochs}')

      # Initialize epoch loss (mean absolute value)
      epoch_loss = 0
      
      # Iterate over all items
      for input_x, reference_y in perceptron._get_item(X, y):
        
        # Make prediction
        predicted_y = self.predict(input_x)[0] # 1-length array

        # Calculate error (loss)
        loss = reference_y - predicted_y

        # Add to epoch loss
        epoch_loss += abs(loss)/num_items

        # Update weights
        self._apply_delta_rule(input_x, loss, learning_rate)
      
      # Log epoch loss
      iterator.set_postfix({'loss': epoch_loss})

      # Append history
      history['loss'].append(epoch_loss)

      # Stop Threshold
      if epoch_loss < stop_threshold:
        print(f'>> Loss met stop condition (at epoch {epoch+1}): loss = {epoch_loss} < {stop_threshold}')
        break

    return history

  def _add_bias_term(self, X: np.ndarray, value = 1.0, is_batch = True) -> np.ndarray:
    '''Add bias term to X values, ie:
       Given (x_i) in = [1, 0] -> out: [value, 1, 0]

      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        value (float): Bias factor value. Default = 1.0
        is_batch (bool): Boolean to explicity that is or not a batch of items

      Returns:
        X (np.ndarray) with the bias term concatenated, eg:
    '''

    if is_batch:
      # Get batch_size
      batch_size = X.shape[0]

      # Add bias term
      X = np.concatenate([value*np.ones((batch_size, 1)), X], axis=1)
    
    else:
      # Add bias term
      X = np.concatenate([[value], X])

    return X

  def _prepare_input(self, X: np.ndarray) -> np.ndarray:
    '''Prepare input X
       
       1. Add batch dimension (if applies)
       2. Add bias term

      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
    '''

    # Make sure is a np.ndarray
    X = np.array(X)

    # Verify if it is sigle item and batch it
    if len(X.shape) == 1:
      # Add batch dimension
      X = np.expand_dims(X, 0)

    # Add bias term
    X = self._add_bias_term(X)

    return X


  def predict(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''
    
    # Prepare input data
    X = self._prepare_input(X)

    # Inner product of inputs and weigths (net)
    net = self.weights * X # multiplication
    net = np.sum(net, axis = 1) # sum reduction

    # Apply activation
    output = self.activation(net)

    return output

### Warmup Problem (AND Operator)

#### Dataset

In [6]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 0, 0, 1])

#### Perceptron with zero weight initialization

In [7]:
# Initialize Model
perceptron = Perceptron(init_rule='zero', activation=f_step1)

# Train Model
perceptron.fit(X, y, max_epochs=10, learning_rate=0.5)

  0%|          | 0/10 [00:00<?, ?it/s]

>> Loss met stop condition (at epoch 6): loss = 0.0 < 1e-06


{'loss': [0.25, 0.75, 0.75, 0.5, 0.25, 0.0]}

In [8]:
# Make predictions
perceptron.predict(X)

array([0., 0., 0., 1.])

In [9]:
# Final weights
perceptron.weights

array([-1. ,  1. ,  0.5])

#### Perceptron with random weight initialization

In [10]:
# Initialize Model
perceptron = Perceptron(init_rule='rand', activation=f_step1)

# Train Model
perceptron.fit(X, y, max_epochs=10, learning_rate=0.5)

  0%|          | 0/10 [00:00<?, ?it/s]

>> Loss met stop condition (at epoch 2): loss = 0.0 < 1e-06


{'loss': [0.25, 0.0]}

In [11]:
# Make predictions
perceptron.predict(X)

array([0., 0., 0., 1.])

In [12]:
# Final weights
perceptron.weights

array([-0.48519715,  0.4682575 ,  0.09218438])

### Binary Digit Recognition (A and A invertido)

#### Dataset

##### Method to generate data

In [13]:
def gen_A(kind: str = 'normal', noise_intensity: float = 0.1, flatten: bool = True) -> tuple:
  '''Generates a 2D array representing A (with noise), eg (inverted):

    A (inverted):
     [[+1, -1, -1, -1, +1],
      [+1, -1, -1, -1, +1],
      [-1, +1, +1, +1, -1],
      [-1, +1, -1, +1, -1],
      [-1, -1, +1, -1, +1]]

    If flatten = True, returns a flatten 1D array

    Params:
      kind (str): type of A to be generated 'normal' or 'inverted'
      noise_intensity (float): probability to apply on every 'point' of A to oposite
        or to keep the value
      flatten (bool): False to return a 2D matrix of A or its True to return its
        flatten 1D representation
    
    Returns:
      tuple: (
        A (np.ndarray): A matrix (2D or 1D) rerpesentation,
        target (float): label of A, -1.0 if it's 'inverted', +1 if it's 'normal'
      )
  '''

  assert kind in ['normal', 'inverted'], "'kind' must be 'normal' or 'inverted'"

  # Build A and its target class
  if kind == 'inverted':
    A = np.array([[+1, -1, -1, -1, +1],
                  [+1, -1, -1, -1, +1],
                  [-1, +1, +1, +1, -1],
                  [-1, +1, -1, +1, -1],
                  [-1, -1, +1, -1, -1]])
    target = -1.0
  
  elif kind == 'normal':
    A = np.array([[-1, -1, +1, -1, -1],
                  [-1, +1, -1, +1, -1],
                  [-1, +1, +1, +1, -1],
                  [+1, -1, -1, -1, +1],
                  [+1, -1, -1, -1, +1]])
    target = 1.0

  # Add noise
  for row in range(len(A)):
    for col in range(len(A[row])):
      
      # If rand less then noise, add noise
      if np.random.rand() < noise_intensity:
        A[row, col] = -A[row, col]
  
  # Transform 2D array into a flatten 1D array
  if flatten:
    A = A.reshape(-1)

  return A, target

##### Visualize Data

In [14]:
noise_intensity = 0.1
A, target = gen_A(kind="normal", flatten=False, noise_intensity=noise_intensity)

fig = px.imshow(A)
fig.update_layout(title=f"Target class: {target} (Sample) | Noise = {noise_intensity}")

In [15]:
noise_intensity = 0.1
A, target = gen_A(kind="inverted", flatten=False, noise_intensity=noise_intensity)

fig = px.imshow(A)
fig.update_layout(title=f"Target class: {target} (Sample) | Noise = {noise_intensity}")

##### Prepare Dataset

In [16]:
# Generate Data
X = []
y = []

num_samples = 50
noise_intensity = 0.1

for i in range(num_samples//2):

  # Gen A normal
  A, target = gen_A(kind='normal', flatten=True, noise_intensity=noise_intensity)

  # Add data
  X.append(A)
  y.append(target)

  # Gen A inverted
  A, target = gen_A(kind='inverted', flatten=True, noise_intensity=noise_intensity)

  # Add data
  X.append(A)
  y.append(target)

X = np.array(X)
y = np.array(y)

X.shape, y.shape

((50, 25), (50,))

#### Perceptron with zero weight initialization

In [17]:
# Initialize Model
perceptron = Perceptron(input_size=len(X[0]), init_rule='zero', activation=f_step2)

# Train Model
perceptron.fit(X, y)

  0%|          | 0/10 [00:00<?, ?it/s]

>> Loss met stop condition (at epoch 2): loss = 0.0 < 1e-06


{'loss': [0.08, 0.0]}

In [18]:
# Predict Data (A normal)
A, _ = gen_A(kind='normal', noise_intensity=0.1)

perceptron.predict(A)

array([1.])

In [19]:
# Predict Data (A inverted)
A, _ = gen_A(kind='inverted', noise_intensity=0.1)

perceptron.predict(A)

array([-1.])

In [20]:
perceptron.weights

array([ 0.,  0.,  0.,  0.,  0., -2.,  0.,  2.,  0.,  2., -2.,  2., -2.,
        0.,  0.,  0.,  0.,  0.,  0., -2.,  2.,  2.,  0., -2.,  0.,  2.])

#### Perceptron with random weight initialization

In [21]:
# Initialize Model
perceptron = Perceptron(input_size=len(X[0]), init_rule='rand', activation=f_step2)

# Train Model
perceptron.fit(X, y)

  0%|          | 0/10 [00:00<?, ?it/s]

>> Loss met stop condition (at epoch 2): loss = 0.0 < 1e-06


{'loss': [0.08, 0.0]}

In [22]:
# Predict Data (A normal)
A, _ = gen_A(kind='normal', noise_intensity=0.1)

perceptron.predict(A)

array([1.])

In [23]:
# Predict Data (A inverted)
A, _ = gen_A(kind='inverted', noise_intensity=0.1)

perceptron.predict(A)

array([-1.])

In [24]:
perceptron.weights

array([-0.39974794,  0.45332252, -0.43463118, -0.44409185,  0.18090765,
       -2.16322029,  0.28909503,  1.69992238, -0.40463027,  2.34308824,
       -1.78789718,  2.26945521, -1.8818649 , -0.46181274, -0.29771359,
       -0.21520466, -0.21009063, -0.00545143, -0.3648994 , -1.62943023,
        1.90745274,  2.14339946, -0.06746718, -1.8557581 ,  0.21576469,
        1.91153735])